In [200]:
# !pip install PyPDF2
# !pip install python-pdfbox
# !pip install resume-parser
# !pip install pyresparser

In [193]:
import spacy
nlp = spacy.load("en_core_web_md")
from spacy_streamlit import visualize_ner, visualize_tokens, visualize_textcat
from spacy.matcher import Matcher

import pandas as pd
import PyPDF2 as pdf
import pdfbox
import re
import string

In [201]:
from pyresparser import ResumeParser
data = ResumeParser('./sample_data/Sohini Banerjee Reddy - Scrum Master.pdf').get_extracted_data()
data


OSError: [E053] Could not read config.cfg from C:\Users\JBanerjee\Anaconda3\lib\site-packages\pyresparser\config.cfg

In [194]:
p = pdfbox.PDFBox()
p.extract_text('./sample_data/Sohini Banerjee Reddy - Scrum Master.pdf')

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [180]:
file = open('./sample_data/Sohini Banerjee Reddy - Scrum Master.pdf', 'rb')

In [181]:
def perform_data_pre_processing(text):

    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    
    text = text.lower()
    text = text.replace('\n', '')
    text = text.replace('|', '')
    text = text.replace(':', '')
    text = text.replace('  ', ' ')
    
    return text

In [182]:
def read_pdf_content(file):
    
    ''' Function to extract pdf data '''
    
    pdf_reader = pdf.PdfFileReader(file)
    page_count = pdf_reader.getNumPages()
    
    if pdf_reader.getIsEncrypted():
        print('File is Encrypted, provide Password')
    else:
        page1      = pdf_reader.getPage(0)
        text_blob  = page1.extractText()
    
    return text_blob

In [183]:
content = read_pdf_content(file)
content

' \n \nSumm\na\nr\ny\n \nof S\nk\ni\nl\nls\n \n \n \n\n \nAgile \nCoach\n \n\n \nEvangelist of agile/scrum \nprinciples and methodology.\n \n\n \nProven ability to work \ncreatively and analytically in a \nproblem\n-\nsolving environment\n \n\n \nMentoring\n \n\n \nTeam Management\n \n\n \nStakeholder \nManagement\n \n\n \nProject \nMangement and \nAllocations\n \n\n \nProject Documentation\n \n\n \nSprint Planning\n, Sprint \nRetrospective\n \n\n \nScrum of Scrums\n \n\n \nCommunity of Practice\n \n\n \nBacklog management\n \n\n \nPM tools: \nClarity PPM,\n \nJIRA\n,\n \nRally,\n \nMicrosoft TFS\n \n\n \nUser stories\n \n\n \nSprint planning\n \n\n \nCoaching agile practices\n \n\n \nRemoving impediment\n \n\n \nRetrospective meetings\n \n\n \nTeam building\n \n\n \nBurndown charts\n \n\n \nQuality control\n \n\n \nCommunication\n \n\n \nDocumentation\n \n\n \nRequirement \nG\nathering\n \n\n \nFunctional Testing\n \n(Automation \n& Manual)\n \n\n \nC, C++, \nJIR\nA,\n \nConfluence,\n

In [184]:
clean_text = perform_data_pre_processing(content)
clean_text

' summary of skills  agile coach evangelist of agile/scrum principles and methodology. proven ability to work creatively and analytically in a problem-solving environment mentoring team management stakeholder management project mangement and allocations project documentation sprint planning, sprint retrospective scrum of scrums community of practice backlog management pm tools clarity ppm, jira, rally, microsoft tfs user stories sprint planning coaching agile practices removing impediment retrospective meetings team building burndown charts quality control communication documentation requirement gathering functional testing (automation & manual) c, c++, jira, confluence, sharepoint, agile, kanban, scrum, sql, business analyst, automation testing (hp qa) certifications certified safe scrum master certified safe agilist  education bachelor in mathematics    sohini banerjee reddy 605-728-9511 sohinireddy79@gmail.com https//www.linkedin.com/in/sohinibanerjeereddy/ i have over 10 years of e

In [189]:
def extract_candidate_name(text):
    
    ''' Function to Extract Candidate Name '''
    
    doc = nlp(text)
    pronoun_list = [token.text for token in doc if (token.pos_ == 'PROPN')]
    
    name      = ' '.join(pronoun_list[0:2]).upper() # Assumed First 2 bytes of Resume is name
    skillset  = ' '.join(pronoun_list[2:]).upper()
    
    return name

In [190]:
def find_required_fields(text):
    
    ''' Function to extract personal details '''
    
    # Data Extraction
    name           = extract_candidate_name(clean_text)
    emails         = re.findall(r"[a-zA-Z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", text)
    github_url     = re.findall(r"github.com[:/](.*)", text)
    phone          = re.findall(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]', text)
    websites       = re.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', text)
    profile_df = pd.DataFrame({
                                    'Name'       : extract_candidate_name(text),
                                    'Email'      : emails, 
                                    'Phone'      : phone, 
                                    'Github'     : 'github.com/'+ ''.join(github_url),
                                    'Other Sites': '/'.join(websites),
                                })
    
    
    return profile_df

In [191]:
find_required_fields(clean_text)

ValueError: All arrays must be of the same length

In [188]:
name, skillset = extract_candidate_name(clean_text)
skillset

'TFS STORIES C++ JIRA SHAREPOINT SCRUM SQL HP QA EDUCATION SOHINI BANERJEE REDDY HTTPS//WWW.LINKEDIN.COM FEB,2020 CHICAGO IL SCRUM NORTH AMERICA EUROPE ASIA PAC . SCRUM JOHNSON MILWAUKEE WI'